In [122]:
import os
import re
import sys

import razdel
import pandas as pd
import numpy as np

sys.path.append('..')

from src import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
filepath = '../data/ruwiki-yo-sentences.txt'
print(f'File size is {os.stat(filepath).st_size / (1024 ** 3):.2f} GB')

File size is 0.00 GB


In [124]:
with open(filepath) as file:
    data = file.readlines()

In [125]:
for sentence in data:
    if 'Помимо фотоэлектрического колориметра' in sentence:
        break

In [126]:
sentence

'Помимо фотоэлектрического колориметра   с непосредственным отсчётом силы тока, распространены компенсационные колориметры, в которых разность сигналов, соответствующих стандартному и измеряемому растворам, сводится к нулю   электрическим или оптическим компенсатором  ; отсчёт в этом случае снимается со шкалы компенсатора.\n'

In [127]:
data[2]

'Грамматическое значение лексем передаётся, как правило, с помощью флексий.\n'

In [128]:
data = pd.DataFrame(data, columns=('text',))
data.head()

,text
0,Является одним из наиболее распространённых яз...
1,Русский является также самым распространённым ...
2,"Грамматическое значение лексем передаётся, как..."
3,"Наряду с синтетическими формами, в русском язы..."
4,В первой половине века этот лингвоним архаизир...


In [129]:
data.text = data.text.str.strip('\n')
data.text = data.text.apply(utils.remove_multiple_spaces)
data['length'] = data.text.apply(len)
data.length.describe()

count    1000.000000
mean      222.726000
std       196.539688
min         2.000000
25%       104.000000
50%       174.000000
75%       278.500000
max      2059.000000
Name: length, dtype: float64

In [130]:
data[data.length.eq(data.length.max())].iloc[0].text[:1000]

'Многочисленные аспекты исторической русистики отражены в исследованиях В. И. Борковского, С. И. Коткова, П. С. Кузнецова, Т. П. Ломтева, С. П. Обнорского, Ф. П. Филина, П. Я. Черных, Л. П. Якубинского; труды по русской диалектологии издают Р. И. Аванесов, К. Ф. Захарова, В. Г. Орлова, К. В. Горшкова, С. В. Бромлей, Л. Н. Булатова, И. Б. Кузьмина; в области фонетики и фонологии появляются работы представителей московской фонологической школы Р. И. Аванесова, А. А. Реформатского, П. С. Кузнецова, М. В. Панова, представителей ленинградской фонологической школы Л. Р. Зиндера, М. И. Матусевич; в области орфоэпии работы Р. И. Аванесова и Н. А. Еськовой; в области интонации работы Е. А. Брызгуновой, Т. М. Николаевой; в области грамматики работы В. Н. Сидорова, П. С. Кузнецова, Н. С. Поспелова, Т. П. Ломтева, А. В. Исаченко, Б. Н. Головина, Ю. С. Маслова, Н. Ю. Шведовой, Д. Н. Шмелёва, А. В. Бондарко, А. А. Зализняка, Ю. Д. Апресяна, И. П. Мучника, Г. А. Золотовой, В. А. Белошапковой, Н. Д. А

In [131]:
def clean_by_length_iqr(
    data: pd.DataFrame, 
    q1: float = 0.25, 
    q3: float = 0.75,
    offset: float = 1.5,
) -> pd.DataFrame:
    q1 = data.length.quantile(q1)
    q3 = data.length.quantile(q3)
    iqr = q3 - q1
    lo = q1 - offset * iqr
    hi = q3 + offset * iqr
    return data[data.length.between(lo, hi)]

In [158]:
clean_data = clean_by_length_iqr(data, q1=0.15, q3=0.5, offset=0.5)
clean_data.length.describe()

count    602.000000
mean     132.270764
std       51.916272
min       27.000000
25%       92.250000
50%      131.000000
75%      175.000000
max      223.000000
Name: length, dtype: float64

In [159]:
clean_data[clean_data.length.eq(clean_data.length.min())]

,text,length
376,"Он ещё молод, время не ушло",27
391,"Василия Чеботарёва , 1861.",27


In [160]:
clean_data[clean_data.length.eq(clean_data.length.max())]

,text,length
206,"М. Элиаде писал:Каждый миф показывает, каким о...",223
236,В рамках фольклористики она может называться м...,223
576,Метод приготовления держался в секрете до моме...,223
932,"Напротив, понятие страны имеет более ёмкое кул...",223


In [161]:
clean_data.iloc[30].text

'В отличие от остальных знаменательных частей речи русского языка, у числительного различительные признаки части речи представлены менее чётко.'

In [162]:
import string

def token_features(text: str) -> pd.Series:
    yo_tokens = []
    num_words = 0
    i = -1
    for i, word in enumerate(razdel.tokenize(text)):
        if word.text not in string.punctuation:
            num_words += 1
        if 'ё' in word.text.lower():
            yo_tokens.append(i)
    return pd.Series(dict(
        yo_tokens=yo_tokens,
        num_words=num_words,
        num_tokens=i + 1,
    ))

In [163]:
token_features(clean_data.iloc[30].text)

yo_tokens     [18]
num_words       18
num_tokens      20
dtype: object

In [164]:
from tqdm import tqdm

tqdm.pandas()

In [165]:
features = clean_data.text.progress_apply(token_features)

100%|██████████| 602/602 [00:00<00:00, 6121.93it/s]


In [166]:
clean_data = pd.concat((clean_data, features), axis=1)
clean_data['num_yo_tokens'] = clean_data.yo_tokens.apply(len)

clean_data.head(10)

,text,length,yo_tokens,num_words,num_tokens,num_yo_tokens
0,Является одним из наиболее распространённых яз...,166,[4],24,25,1
2,"Грамматическое значение лексем передаётся, как...",74,[3],9,12,1
4,В первой половине века этот лингвоним архаизир...,95,[8],13,14,1
5,"Согласно переписи 2002 года, владение русским ...",199,[20],27,32,1
10,"Он используется во всех сферах политической, э...",153,[16],21,25,1
12,Он сравнительно широко распространён и во мног...,69,[3],10,11,1
14,На нём создана богатая оригинальная художестве...,161,[1],21,24,1
16,"В нём выделяются две разновидности, письменная...",61,[1],8,10,1
17,Современный литературный русский язык обрёл св...,196,[4],27,31,1
18,Как предмет научных исследований русский язык ...,134,[6],17,18,1


In [167]:
clean_data['words_to_length_ratio'] = clean_data.num_words / clean_data.length

clean_data.words_to_length_ratio.describe()

count    602.000000
mean       0.133411
std        0.020486
min        0.067669
25%        0.120710
50%        0.131350
75%        0.144385
max        0.233333
Name: words_to_length_ratio, dtype: float64

In [171]:
clean_data[clean_data.words_to_length_ratio.ge(clean_data.words_to_length_ratio.quantile(0.7))].iloc[10].text

'4-я буква форма частиц .Товарный силикагель обычно выпускают в виде зёрен или шаровидных гранул размером от 5 7 до 10 мм.'

In [121]:
# todo: спросить в чатах нет ли какой-то нарезки получше для слипшихся предложений

In [20]:
clean_data[clean_data.num_yo_tokens.gt(1)]

,text,length,yo_tokens,num_words,num_tokens,num_yo_tokens
16,В этот же период литовские князья из династии ...,233,"[10, 25]",32,34,2
25,Вос­ста­но­ви­тель­ный пе­ри­од в Литве был ос...,164,"[5, 9]",3,16,2
34,"Ещё относительно дешёвая, и хорошо образованна...",91,"[0, 2]",13,17,2
43,"Г. Форстер, Ж. Жилибер, И. Франк и многие друг...",106,"[14, 15]",15,21,2
44,Основание Вильнюсского университета в 1579 год...,183,"[12, 22]",24,25,2
...,...,...,...,...,...,...
999751,Почётный доктор богословия «Honoris causa» Кие...,196,"[0, 22]",23,28,2
999756,В 1958—1960 годах учился в Киевской духовной с...,150,"[10, 13]",21,25,2
999758,Учёным советом Киевской духовной академии от 2...,272,"[0, 11, 24, 35]",36,40,4
999775,"Как сообщается, проведённый Генеральной прокур...",160,"[3, 8]",19,23,2


In [21]:
clean_data.loc[999758]

text             Учёным советом Киевской духовной академии от 2...
length                                                         272
yo_tokens                                          [0, 11, 24, 35]
num_words                                                       36
num_tokens                                                      40
num_yo_tokens                                                    4
Name: 999758, dtype: object